# Question 1 Solution

In [1]:
%pip install pandas numpy matplotlib seaborn



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

# Load the dataset
df = pd.read_csv('data/train.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


## a) Look for the missing values in all the columns and either impute them or drop them. Justify your action.

In [3]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

# Calculate percentage of missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
print("\nPercentage of missing values:")
print(missing_percent)

Missing values per column:
Unnamed: 0              0
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  38
New_Price            5032
Price                   0
dtype: int64

Percentage of missing values:
Unnamed: 0            0.000000
Name                  0.000000
Location              0.000000
Year                  0.000000
Kilometers_Driven     0.000000
Fuel_Type             0.000000
Transmission          0.000000
Owner_Type            0.000000
Mileage               0.034206
Engine                0.615700
Power                 0.615700
Seats                 0.649906
New_Price            86.061228
Price                 0.000000
dtype: float64


**Justification:**
- `New_Price`: This column has over 86% missing values. Imputing such a large portion of data would introduce significant bias and is not reliable. Therefore, we will drop this column.
- `Mileage`, `Engine`, `Power`, `Seats`: These columns have a very small percentage of missing values (less than 1%). Dropping these few rows will not significantly impact the dataset size or analysis.

In [4]:
# Drop New_Price column
if 'New_Price' in df.columns:
    df = df.drop(columns=['New_Price'])

# Drop rows with missing values for other columns
df = df.dropna()

# Verify missing values are handled
print(df.isnull().sum())

Unnamed: 0           0
Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64


## b) Remove the units from some of the attributes and only keep the numerical values.

In [5]:
# Function to extract numerical values
def clean_currency(x):
    if isinstance(x, str):
        return float(x.split()[0])
    return x

# Clean Mileage
# Splitting by space and taking the first part. Handling NaN just in case.
df['Mileage'] = df['Mileage'].astype(str).apply(lambda x: float(x.split()[0]) if 'km' in x else np.nan)

# Clean Engine
df['Engine'] = df['Engine'].astype(str).apply(lambda x: float(x.split()[0]) if 'CC' in x else np.nan)

# Clean Power
# Power sometimes has 'null bhp'
df['Power'] = df['Power'].astype(str).apply(lambda x: float(x.split()[0]) if 'bhp' in x and x.split()[0] != 'null' else np.nan)

# Drop any rows that became NaN during conversion (e.g. 'null bhp')
df = df.dropna()

print(df[['Mileage', 'Engine', 'Power']].head())
print(df.info())

   Mileage  Engine   Power
0    19.67  1582.0  126.20
1    13.00  1199.0   88.70
2    20.77  1248.0   88.76
3    15.20  1968.0  140.80
4    23.08  1461.0   63.10
<class 'pandas.core.frame.DataFrame'>
Index: 5807 entries, 0 to 5846
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5807 non-null   int64  
 1   Name               5807 non-null   object 
 2   Location           5807 non-null   object 
 3   Year               5807 non-null   int64  
 4   Kilometers_Driven  5807 non-null   int64  
 5   Fuel_Type          5807 non-null   object 
 6   Transmission       5807 non-null   object 
 7   Owner_Type         5807 non-null   object 
 8   Mileage            5807 non-null   float64
 9   Engine             5807 non-null   float64
 10  Power              5807 non-null   float64
 11  Seats              5807 non-null   float64
 12  Price              5807 non-null   float64
dtypes: float64(

## c) Change the categorical variables (“Fuel_Type” and “Transmission”) into numerical one hot encoded value.

In [6]:
# One hot encoding for Fuel_Type and Transmission
# Using pd.get_dummies. 
# We convert boolean result to integer (0 and 1) for better readability/compatibility
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True, dtype=int)

print(df.head())

   Unnamed: 0                              Name    Location  Year  \
0           1  Hyundai Creta 1.6 CRDi SX Option        Pune  2015   
1           2                      Honda Jazz V     Chennai  2011   
2           3                 Maruti Ertiga VDI     Chennai  2012   
3           4   Audi A4 New 2.0 TDI Multitronic  Coimbatore  2013   
4           6            Nissan Micra Diesel XV      Jaipur  2013   

   Kilometers_Driven Owner_Type  Mileage  Engine   Power  Seats  Price  \
0              41000      First    19.67  1582.0  126.20    5.0  12.50   
1              46000      First    13.00  1199.0   88.70    5.0   4.50   
2              87000      First    20.77  1248.0   88.76    7.0   6.00   
3              40670     Second    15.20  1968.0  140.80    5.0  17.74   
4              86999      First    23.08  1461.0   63.10    5.0   3.50   

   Fuel_Type_Petrol  Transmission_Manual  
0                 0                    1  
1                 1                    1  
2          

## d) Create one more feature and add this column to the dataset.

In [7]:
# Calculate Current Age
current_year = datetime.datetime.now().year
df['Current_Age'] = current_year - df['Year']

print(df[['Year', 'Current_Age']].head())

   Year  Current_Age
0  2015           10
1  2011           14
2  2012           13
3  2013           12
4  2013           12


## e) Perform select, filter, rename, mutate, arrange and summarize with group by operations.

In [8]:
# 1. Select specific columns
selected_df = df[['Name', 'Location', 'Current_Age', 'Price']]

# 2. Filter: Cars with Price > 10 Lakh
filtered_df = selected_df[selected_df['Price'] > 10]

# 3. Rename columns
renamed_df = filtered_df.rename(columns={'Name': 'Car_Name', 'Price': 'Price_Lakhs'})

# 4. Mutate: Add a new column (e.g., Price in Thousands)
mutated_df = renamed_df.assign(Price_Thousands = renamed_df['Price_Lakhs'] * 100)

# 5. Arrange: Sort by Price descending
arranged_df = mutated_df.sort_values(by='Price_Lakhs', ascending=False)

# 6. Summarize with Group By: Average Price by Location
summary_df = arranged_df.groupby('Location')['Price_Lakhs'].mean().reset_index()

print("Summary of Average Price by Location (for cars > 10 Lakh):")
print(summary_df)

Summary of Average Price by Location (for cars > 10 Lakh):
      Location  Price_Lakhs
0    Ahmedabad    18.717193
1    Bangalore    27.154662
2      Chennai    23.382222
3   Coimbatore    27.819733
4        Delhi    22.568774
5    Hyderabad    24.567747
6       Jaipur    18.392500
7        Kochi    24.788118
8      Kolkata    21.330943
9       Mumbai    22.643264
10        Pune    19.962019
